# MC3E Radar Matching

This script computes Z from the nearest-neighboring gates of a ground-based radar with the GPS coordinates of an aircraft carrying microphysical instruments.

In [1]:
%config InlineBackend.figure_formats = {'png', 'retina'}
%pylab inline
import pyart
import awot
import os, glob
import awot
import pandas as pd
from awot.graph import create_basemap, FlightLevel, RadarHorizontalPlot
from netCDF4 import date2num, num2date
import datetime
from csu_radartools import (csu_fhc, csu_liquid_ice_mass, csu_blended_rain, 
                            csu_dsd, csu_kdp, csu_misc)
import copy
from numba import jit

Populating the interactive namespace from numpy and matplotlib

## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def extract_unmasked_data(radar, field, bad=-32768):
    """Simplify getting unmasked radar fields from Py-ART"""
    return radar.fields[field]['data'].filled(fill_value=bad)

def add_field_to_radar_object(field, radar, field_name='FH', units='unitless', 
                              long_name='Hydrometeor ID', standard_name='Hydrometeor ID',
                              dz_field='ZC'):
    """
    Adds a newly created field to the Py-ART radar object. If reflectivity is a masked array,
    make the new field masked the same as reflectivity.
    """
    masked_field = np.ma.asanyarray(field)
    fill_value = -32768
    if hasattr(radar.fields[dz_field]['data'], 'mask'):
        setattr(masked_field, 'mask', radar.fields[dz_field]['data'].mask)
        fill_value = radar.fields[dz_field]['_FillValue']
    field_dict = {'data': masked_field,
                  'units': units,
                  'long_name': long_name,
                  'standard_name': standard_name,
                  '_FillValue': fill_value}
    radar.add_field(field_name, field_dict, replace_existing=True)
    return radar

@jit
def rolling_window(a, window):
    """ create a rolling window object for application of functions
    eg: result=np.ma.std(array, 11), 1)"""
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1], )
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

@jit
def texture(myradar, var):
    """Determine a texture field using an 11pt stdev
    texarray=texture(pyradarobj, field)
    """
    fld = myradar.fields[var]['data']
#    print(fld.shape)
    tex = np.ma.zeros(fld.shape)
    for timestep in range(tex.shape[0]):
        ray = np.ma.std(rolling_window(fld[timestep, :], 5), 1)
        tex[timestep, 2:-2] = ray
        tex[timestep, 0:2] = np.ones(1) * ray[0]
        tex[timestep, -2:] = np.ones(2) * ray[-1]
    return tex

@jit
def do_auto_qc(iradar):
    dzN = extract_unmasked_data(iradar, 'reflectivity')
    dpN = extract_unmasked_data(iradar, 'differential_phase')
    # Range needs to be supplied as a variable, and it needs to be the same shape as dzN, etc.
    rng2d, az2d = np.meshgrid(iradar.range['data'], iradar.azimuth['data'])
    kdN, fdN, sdN = csu_kdp.calc_kdp_bringi(dp=dpN, dz=dzN, rng=rng2d/1000.0, thsd=20., gs=125.0, window=1.5,nfilter=1)
    
    iradar = add_field_to_radar_object(kdN, iradar, field_name='KDP', units='deg/km', 
                                   long_name='Specific Differential Phase',
                                   standard_name='Specific Differential Phase', 
                                   dz_field='reflectivity')
    iradar = add_field_to_radar_object(fdN, iradar, field_name='FDP', units='deg', 
                                   long_name='Filtered Differential Phase',
                                   standard_name='Filtered Differential Phase', 
                                   dz_field='reflectivity')

    iradar = add_field_to_radar_object(sdN, iradar, field_name='SDP', units='deg**2/km**2', 
                                   long_name='Sigma Differential Phase',
                                   standard_name='Sigma Differential Phase', 
                                   dz_field='reflectivity')

    bad=-32768
    iradar.fields['KDP']['data']=np.ma.masked_equal(iradar.fields['KDP']['data'],bad)
    iradar.fields['SDP']['data']=np.ma.masked_equal(iradar.fields['SDP']['data'],bad)
    iradar.fields['FDP']['data']=np.ma.masked_equal(iradar.fields['FDP']['data'],bad)
    
    cor_z = copy.deepcopy(iradar.fields['reflectivity'])
    cor_z['data'] = np.ma.masked_where(radar.fields['cross_correlation_ratio']['data'] < 0.75, cor_z['data'])
    cor_z['data'] = np.ma.masked_where(radar.fields['SDP']['data'] > 12, cor_z['data'])
    cor_z['data'] = np.ma.masked_where(radar.fields['differential_reflectivity']['data'] < -2., cor_z['data'])
    cor_z['data'] = np.ma.masked_where(radar.fields['differential_reflectivity']['data'] > 3., cor_z['data'])
#    cor_z['data'] = np.ma.masked_where(radar.fields['normalized_coherent_power']['data'] < .3, cor_z['data'])

    
    cor_z['least_significant_digit'] = 2
    cor_z['valid_max'] = 80.0
    cor_z['valid_min'] = -30.0
    cor_z['standard_name'] = 'corrected_reflectivity'
    cor_z['long_name'] = 'corrected_reflectivity'
    cor_z['least_significant_digit'] = 2
    cor_z['units'] = 'dBZ'

    iradar.add_field('CORDBZ',cor_z)
    
    Ztext = copy.deepcopy(iradar.fields['CORDBZ'])
    Ztext['data'] = texture(iradar,'CORDBZ')
    Ztext['least_significant_digit'] = 2
    Ztext['valid_max'] = 80.0
    Ztext['valid_min'] = -30.0
    Ztext['standard_name'] = 'corrected_reflectivity_texture'
    Ztext['long_name'] = 'corrected_reflectivity_texture'
    Ztext['least_significant_digit'] = 2
    Ztext['units'] = 'dBZ'
    iradar.add_field('Ztext',Ztext)
    
    iradar.fields['CORDBZ']['data'] = np.ma.masked_where(radar.fields['Ztext']['data'] > 7., cor_z['data'])

    fields=['cross_correlation_ratio','differential_reflectivity','KDP']
    for ifield in fields:
        iradar.fields[ifield]['data']=np.ma.masked_where(np.ma.getmask(iradar.fields['CORDBZ']['data']), iradar.fields[ifield]['data'])
#    pyart.io.cfradial.write_cfradial('test.cfradial.nc',iradar, format='NETCDF4')

In [3]:
# folders=glob.glob("/data/snesbitt/h/snesbitt/gpm/gcpex/cloud_microphysics_Citation/UND_cloud_microphysics/data/QC_Processed/*")
path = '/data/gpm/a/shared/finlon2/mc3e/flight_data/'
folders=glob.glob('/data/gpm/a/shared/finlon2/mc3e/flight_data/*')

# for flights in folders[2:]:
#     print(flights.split('/')[-1])
#     str1=flights.split('/')[-1]
#     str2=str1[0:4]+'_'+str1[4:6]+'_'+str1[6:8]+'_'+str1[9:11]+'_'+str1[11:13]+'_'+str1[13:15]
#     str3=str1[0:8]
#     print str3
for flights in folders[0:]:
    print(flights.split('/')[-1])
    str1=flights.split('/')[-1]
    str3=str1[0:4]+str1[5:7]+str1[8:10]
    print(str3)


    # Set the project name
#     Project="GCPEx"
    Project="MC3E"
    # Set the path for data file
#     flname="/data/snesbitt/h/snesbitt/gpm/gcpex/cloud_microphysics_Citation/UND_cloud_microphysics/data/QC_Processed/" + str1 + '/' + str2 + ".gcpex"
    flname='/data/gpm/a/shared/finlon2/mc3e/flight_data/{}'.format(str1)

    fl = awot.io.flight.read_nasa_ames(flname,platform='citation',verbose=False)
    startdate=datetime.datetime.strftime(fl['time']['data'][0],'%Y%m%d')
    enddate=datetime.datetime.strftime(fl['time']['data'][-1],'%Y%m%d')

    rfiles = []
#     rfiles.append(glob.glob('/data/snesbitt/h/snesbitt/gpm/gcpex/WKR/iri/SECTOR/'+str3+'/*.iri'))
#     rfiles.append(glob.glob('/data/snesbitt/h/snesbitt/gpm/gcpex/WKR/iri/CONVOL/'+str3+'/*.iri'))
#     rfiles=rfiles[0]+rfiles[1]
    rfiles.append(glob.glob('/data/gpm/a/shared/finlon2/mc3e/radar_data/'+str3+'/KVNX4/*V06'))
    rfiles=rfiles[0]
    rfiles = sorted(rfiles)
    rfiles

    rflist = []
    volstart = []
    for filen, filer in enumerate(rfiles):
#         radar=pyart.io.read_sigmet(filer)
        radar = pyart.io.read_nexrad_archive(filer)
        timedata=radar.time['data']
        timeunits=radar.time['units']
        datet=num2date(timedata,units = timeunits)
        if (min(datet) > min(fl['time']['data'])) & (max(datet) < max(fl['time']['data'])):
            radar=radar.extract_sweeps(np.arange(0,radar.nsweeps-1))
            do_auto_qc(radar)
            rflist.append(radar)
            volstart.append(min(fl['time']['data']))
            print("File {} is in time bounds".format(filer))
        else:
            print("File {} is OUT of time bounds".format(filer))
    print('Matching reflectivities from {} volume scans.'.format(len(rflist)))

    rflist2=[x for y, x in sorted(zip(volstart, rflist))]
    rflist2

    und_set = [fl['time']['data'][0].isoformat(), fl['time']['data'][-1].isoformat()]
    rmatch = awot.util.RadarMatch(fl, rflist2,
                                       mask_above=1E5,
                                       start_time=und_set[0], end_time=und_set[1],
                                       field_match_dict=['CORDBZ','SDP','KDP','differential_reflectivity',
                                                         'normalized_coherent_power','velocity','cross_correlation_ratio'])

    kdWKR = rmatch.kdtree(leafsize=16, query_k=8, Barnes=True, K_d=5e2, Zfield='CORDBZ')
    ind1 = kdWKR.matchinfo['match_indicies'][:,0]
    ind2 = kdWKR.matchinfo['match_indicies'][:,1]
    ind3 = kdWKR.matchinfo['match_indicies'][:,2]
    ind4 = kdWKR.matchinfo['match_indicies'][:,3]
    ind5 = kdWKR.matchinfo['match_indicies'][:,4]
    ind6 = kdWKR.matchinfo['match_indicies'][:,5]
    ind7 = kdWKR.matchinfo['match_indicies'][:,6]
    ind8 = kdWKR.matchinfo['match_indicies'][:,7]
    
    data=pd.DataFrame({'quality_controlled_reflectivity':kdWKR.data['CORDBZ']['data'],
                       'specific_differential_phase':kdWKR.data['KDP']['data'],
                       'standard_deviation_differential_phase':kdWKR.data['SDP']['data'],
                       'standard_deviation_reflectivity':kdWKR.data['Ztext']['data'],
                       'cross_correlation_ratio':kdWKR.data['cross_correlation_ratio']['data'],
                       'differential_reflectivity':kdWKR.data['differential_reflectivity']['data'],
                       'reflectivity':kdWKR.data['reflectivity']['data'],
                       'gate1_ind':ind1,'gate2_ind':ind2,'gate3_ind':ind3,'gate4_ind':ind4,
                       'gate5_ind':ind5,'gate6_ind':ind6,'gate7_ind':ind7,'gate8_ind':ind8,
                       'radar_time (epoch)':kdWKR.matchinfo['match_time']},index=kdWKR.flight['time']['data'])

    data.to_csv('/data/gpm/a/shared/finlon2/mc3e/mD_analysis/'+str3+'/RadarMatchNew4.'+str3+'.csv')

2011_05_20_12_54_03_v2.mc3e
20110520
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_163248_V06 is in time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_163746_V06 is in time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_164245_V06 is in time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_164743_V06 is in time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_165241_V06 is in time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_165740_V06 is in time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_170238_V06 is OUT of time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_170737_V06 is OUT of time bounds
File /data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX4/KVNX20110520_171235_V06 is OUT of time bounds
File /

ValueError: need more than 1 value to unpack

In [ ]:
# rfile = '/data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX/KVNX20110520_032129_V06'
# # rfile = '/data/gpm/a/shared/finlon2/mc3e/radar_data/20110520/KVNX/KVNX20110520_032549_V06'
# radar = pyart.io.read_nexrad_archive(rfile)
# radar=radar.extract_sweeps(np.arange(0,radar.nsweeps-1))
# do_auto_qc(radar)
# list(radar.fields)

In [ ]:
# display=pyart.graph.RadarMapDisplay(radar)
# display.plot_ppi('reflectivity',6, vmin=-20., vmax=70.)
# display.set_limits(xlim=[-300,300], ylim=[-300,300])

In [ ]:
# display=pyart.graph.RadarMapDisplay(radar)
# display.plot_ppi('CORDBZ',6, cmap=pyart.graph.cm.NWSRef, vmin=-20., vmax=70.)
# display.set_limits(xlim=[-300,300], ylim=[-300,300])

In [4]:
fl
# radarobj = rflist2[2]
# print(radarobj.time['units'][14:])
# print(radarobj.fields['KDP'])
# # partStr = radar.time['units'][14:]; print(partStr)

{'Conc_2DC': {'data': masked_array(data = [ 0.  0.  0. ...,  0.  0.  0.],
               mask = False,
         fill_value = 9999.999999),
  'long_name': 'Number concentration of droplets based on the 2-DC Probe measurements [#/cm^3]',
  'standard_name': 'Conc_2DC',
  'units': ' '},
 'Conc_CDP': {'data': masked_array(data = [ 0.  0.  0. ...,  0.  0.  0.],
               mask = False,
         fill_value = 9999.999999),
  'long_name': 'Number Concentration of Droplets Based on the Cloud Droplet Probe [#/cc]',
  'standard_name': 'Conc_CDP',
  'units': ' '},
 'Conc_CPC': {'data': masked_array(data = [0.06 -- 0.0 ..., 15790.0 -- --],
               mask = [False  True False ..., False  True  True],
         fill_value = 999999.9999),
  'long_name': 'Total Concentration from CPC [#/cm^3]',
  'standard_name': 'Conc_CPC',
  'units': ' '},
 'Deff_2DC': {'data': masked_array(data = [-- -- -- ..., -- -- --],
               mask = [ True  True  True ...,  True  True  True],
         fill_value = 

In [ ]:
strVar='2011-05-20T12:59:31Z'
print(strVar[0:4]*strVar[5:7])